In [12]:
import pandas as pd
import numpy as np
import dask
from sklearn.preprocessing import MinMaxScaler

import data

In [13]:
inputfile = '/Users/JuJohnson/git/CMS-2019-Preprocessing/data/partb/medicare-partb-aggregated-2013-2019.csv.gz'

In [14]:
usecols = data.get_columns(inputfile)
df = pd.read_csv(inputfile, usecols=usecols, nrows=1_000_000)
onehot_cols = data.get_onehot_columns(inputfile)
df = pd.get_dummies(df, columns=onehot_cols)
y, x = df["exclusion"], df.drop(columns=["exclusion"])

columns = x.columns
scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=columns)
x.shape

(1000000, 134)

In [20]:
from tbfs.multiprocess.ranker import TBFSRanker

In [15]:
thresholds = [
  *np.arange(0, 0.01, 0.0001),
]

print(len(thresholds))

100


In [21]:
%%time

ranker = TBFSRanker(thresholds=thresholds, n_jobs=10)
ranker.fit(x, y)
top_5_v1 = ranker.top_k_features_by_metric(10, 'auc')

CPU times: user 27.3 s, sys: 4.02 s, total: 31.3 s
Wall time: 28min 43s


In [16]:
from tbfs.ranker import TBFSRanker

In [17]:
%%time

ranker = TBFSRanker(thresholds=thresholds)
ranker.fit(x, y)
top_5_v2 = ranker.top_k_features_by_metric(10, 'auc')

CPU times: user 21min 58s, sys: 3min 52s, total: 25min 50s
Wall time: 25min 53s
